Use this script to update csv files with existing price history for any coin on binance.
Call functie by : data = get_all_binance("ticker", "interval", save=True)

In [1]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
import Bkeys
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser

### You need API keys in order to use this

binance_api_key = Bkeys.Pkey
binance_api_secret = Bkeys.Skey

### CONSTANTS
binsizes = {"1m": 1, "5m": 5,"15m": 15, "1h": 60, "4h": 240, "1d": 1440}
batch_size = 750
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('16 Nov 2020', '%d %b %Y')
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('16 Nov 2020', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [58]:
data = get_all_binance("ETHBTC", "15m", save=True)

# Additonal modifications to dataframe
data_df = data_df['2020-11-21 23:00:00' : '2021-04-22 00:00:00']
data.reset_index(inplace=True)
data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M')

#Make sure to save in seperate csv-file


All caught up..!
